In [1]:
# import findspark
import findspark
# initialize findspark with spark directory
findspark.init("C:\Program Files\Spark\spark-3.3.1-bin-hadoop3")
#findspark.init("/Users/wouterdewitte/spark/")
# import pyspark
import pyspark
# create spark context
sc = pyspark.SparkContext()
# create spark session 
spark = pyspark.sql.SparkSession(sc)

In [2]:
# import packages
import os 
import pickle
import re
from datetime import datetime
import requests
import pytz
import emojis
import pandas as pd
import numpy as np
import ast
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import array_contains
import matplotlib.pyplot as plt 
import emojis

import pandas as pd
import numpy as np
import ast

import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
import csv
import pandas as pd
import os
import json
import plotly.express as px
from pandas.tseries.holiday import nearest_workday, \
    AbstractHolidayCalendar, Holiday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, \
    USMemorialDay, USLaborDay, USThanksgivingDay
from sklearn.metrics import roc_curve, auc
from sklearn.datasets import make_classification

## General

In this notebook we will buid a model that predicts if the trend of a certain topic goes up or down on a certain day based on Twitter data of that day.

## 1. Import Data

### 1.1 Google Trends

In [32]:
# read trend data 
trend = spark.read.csv(".././../data/Google_trends/daily_trends.csv", header=True, inferSchema=True, sep=';')

In [33]:
trend 

DataFrame[date: timestamp, dependent_vegan: int]

In [37]:
from pyspark.sql.window import Window

w = Window().partitionBy().orderBy(col("date"))
trend.withColumn("dependent_vegan", lag("dependent_vegan", -1, 0).over(w)).show(100)

+-------------------+---------------+
|               date|dependent_vegan|
+-------------------+---------------+
|2021-10-04 00:00:00|              1|
|2021-10-05 00:00:00|              1|
|2021-10-06 00:00:00|              1|
|2021-10-07 00:00:00|              1|
|2021-10-08 00:00:00|              1|
|2021-10-09 00:00:00|              0|
|2021-10-10 00:00:00|              0|
|2021-10-11 00:00:00|              0|
|2021-10-12 00:00:00|              0|
|2021-10-13 00:00:00|              0|
|2021-10-14 00:00:00|              0|
|2021-10-15 00:00:00|              1|
|2021-10-16 00:00:00|              1|
|2021-10-17 00:00:00|              0|
|2021-10-18 00:00:00|              1|
|2021-10-19 00:00:00|              0|
|2021-10-20 00:00:00|              0|
|2021-10-21 00:00:00|              1|
|2021-10-22 00:00:00|              1|
|2021-10-23 00:00:00|              1|
|2021-10-24 00:00:00|              0|
|2021-10-25 00:00:00|              0|
|2021-10-26 00:00:00|              0|
|2021-10-27 

In [35]:
# create SQL view
trend.createOrReplaceTempView("trendSQL")

The binary variable indicates if the trend goes up or down.

### 1.2 Twitter

In [ ]:
# define data dir
data_dir = "../../data/Topic/"

# get all twitter files
tweet_files = [os.path.join(data_dir, obs) for obs in os.listdir(data_dir)] 

In [ ]:
# import twitter data 
#twitter_df = spark.read.json(tweet_files)

In [ ]:
list_hashtags = ["vegan"]

data_dir = ".././../data/Topic/"
tweet_files = [os.path.join(data_dir, obs) for obs in os.listdir(data_dir)]
files_hashtags = [file for file in tweet_files if (file.find(list_hashtags[0]) != -1)]             
twitter_df = spark.read.option("multiline","true").json(files_hashtags) 
twitter_df.count()

In [ ]:
# select interesting features
twitter_df = twitter_df.select(F.col('user.name'),
                                F.col('user.screen_name'),
                                F.col('user.followers_count'),
                                F.col('user.following'),
                                F.col('user.statuses_count'),
                                F.col('user.listed_count'),
                                F.col('created_at'),
                                F.col('full_text'),
                                F.col('entities.hashtags'),
                                F.col('favorite_count'),
                                F.col('retweet_count'),
                                F.col('user.friends_count'))

## 2. Data Preprocessing

### 2.1 Check time period

In [ ]:
# function to convert Twitter date string format
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

# UDF declaration
date_udf = F.udf(getDate, StringType())

# apply udf
twitter_df = twitter_df.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

In [ ]:
# get first post
first_post = F.min('post_created_at').alias('earliest')
# get latest post
latest_post = F.max('post_created_at').alias('latest')
# show tweet period in our dataset
twitter_df.select(first_post, latest_post).show()

### 2.2 Remove retweets and duplicates

In [ ]:
# drop all retweets from dataset
no_retweets_df = twitter_df.filter(~F.col("full_text").startswith("RT"))

In [ ]:
# first sort no_retweets_df based on date in chronological order (most recent ones on top)
no_retweets_sorted_df = no_retweets_df.sort("post_created_at", ascending=False)

In [ ]:
# number of observations before dropping duplicates
no_retweets_sorted_df.count()

In [ ]:
# drop duplicates based on tweet text and the profile it was posted from
final_no_duplicates_df = no_retweets_sorted_df.drop_duplicates(["full_text", "screen_name"])

In [ ]:
# number of observations after dropping duplicates
final_no_duplicates_df.count()

In [ ]:
# rename dataframe
final_twitter_df = final_no_duplicates_df

## 3. Independent Variables

For our independent variables we need to design a pipeline that transforms the data into the desired aggregated metrics per day.

### 3.0 Feature Engineering

#### Define Functions

In [ ]:
# define function to count hashtags
def get_hashtags(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "#" in word:
            counter += 1
    return(counter)

In [ ]:
# define function to count mentions
def get_mentions(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "@" in word:
            counter += 1
    return(counter)

In [ ]:
# define function to count exclamation marks
def get_exclamation_marks(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "!" in word:
            counter += 1
    return(counter)

In [ ]:
# define function to count number of emojis used
def emoji_counter(text):
    nr_emojis = emojis.count(text)
    return(nr_emojis)

In [ ]:
# define function to calculate engagement rate
def engagement_rate(favorite_count, retweet_count, followers_count):
    if(followers_count == 0):
        eng_rate = 0
    else:
        eng_rate = (favorite_count + retweet_count)/followers_count
    
    return eng_rate

In [ ]:
# register functions as udf
get_hashtags_UDF = F.udf(get_hashtags, IntegerType())
get_mentions_UDF = F.udf(get_mentions, IntegerType())
get_exclamation_marks_UDF = F.udf(get_exclamation_marks, IntegerType())
emoji_counter_UDF = F.udf(emoji_counter, IntegerType())
engagement_rate_UDF = F.udf(engagement_rate, DoubleType())

In [ ]:
# apply functions to create new features
final_twitter_df = final_twitter_df.withColumn("emoji_count", emoji_counter_UDF("full_text")) \
        .withColumn("text_tokenized", F.split("full_text", " ")) \
        .withColumn("num_words", F.size("text_tokenized")) \
        .withColumn("num_hashtags", get_hashtags_UDF("text_tokenized")) \
        .withColumn("num_mentions", get_mentions_UDF("text_tokenized")) \
        .withColumn("num_exclamation_marks", get_exclamation_marks_UDF("text_tokenized")) \
        .withColumn("engagement_rate", engagement_rate_UDF("favorite_count", "retweet_count", "followers_count"))

#### Text Cleaning

In [ ]:
# filter for english tweets (NOTE: for the assignment you can translate non-english tweets using an API)
final_twitter_df = final_twitter_df.filter(F.col("lang") == "en")

In [ ]:
# check number of observations
final_twitter_df.count()

In [ ]:
# define function to clean text
def clean_text(string):
    
    # define numbers
    NUMBERS = '0123456789'
    PUNCT = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    
    # convert text to lower case
    cleaned_string = string.lower()
    
    # remove URLS
    cleaned_string = re.sub(r'http\S+', ' ', cleaned_string)
    
    # replace emojis by words
    cleaned_string = emojis.decode(cleaned_string)
    cleaned_string = cleaned_string.replace(":"," ").replace("_"," ")
    cleaned_string = ' '.join(cleaned_string.split())
    
    # remove numbers
    cleaned_string = "".join([char for char in cleaned_string if char not in NUMBERS])
    
    # remove punctuation
    cleaned_string = "".join([char for char in cleaned_string if char not in PUNCT])
    
    # remove words conisting of one character (or less)
    cleaned_string = ' '.join([w for w in cleaned_string.split() if len(w) > 1])
    
    # return
    return(cleaned_string)

In [ ]:
# convert to udf
clean_text_udf = F.udf(clean_text, StringType())

In [ ]:
# clean string
final_twitter_df = final_twitter_df.withColumn("cleaned_text", clean_text_udf(F.col("full_text")))

In [ ]:
# check
pd.set_option('display.max_colwidth', None)
final_twitter_df.select("full_text", "cleaned_text").limit(5).toPandas()

#### Sentiment

VADER sentimental analysis relies on a dictionary that maps lexical features to emotion intensities known as sentiment scores. The sentiment score of a text can be obtained by summing up the intensity of each word in the text.

In [ ]:
#using the vaderSentiment package 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [ ]:
# define the function to extract the sentiment
def get_sentiment(sentence):

    # initialize sentiment analyzer
    sid_obj = SentimentIntensityAnalyzer()

    # get sentiment dict
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    # get positive sentiment score
    pos_sentiment = sentiment_dict["pos"]
    
    # return positive sentiment score
    return(pos_sentiment)

get_sentiment_udf = udf(get_sentiment, DoubleType())

In [ ]:
final_twitter_df = final_twitter_df.withColumn("sentiment_vader", get_sentiment_udf(F.col("cleaned_text")))

In [ ]:
final_twitter_df.select("cleaned_text", "sentiment_vader").show()

#### TextBlob

TextBlob returns polarity and subjectivity of a sentence. 

**Polarity** lies between [-1,1],  -1 defines a negative sentiment and 1 defines a positive sentiment.  

**Subjectivity** quantifies the amount of personal opinion and factual information contained in the text. Subjectivity lies between [0,1]. The higher subjectivity means that the text contains personal opinion rather than factual information. 

In [ ]:
#use polarity and subjectivity from TextBlob 
#https://textblob.readthedocs.io/en/dev/
from textblob import TextBlob

# define function to get polarity score of text document
def get_polarity(row):
    textBlob_review = TextBlob(row)
    return textBlob_review.sentiment[0]
# define function to get subjectivity score of text document
def get_subjectivity(row):
    textBlob_review = TextBlob(row)
    return textBlob_review.sentiment[1]
get_polarity_udf = F.udf(get_polarity, DoubleType())
get_subjectivity_udf = F.udf(get_subjectivity, DoubleType())

final_twitter_df = final_twitter_df.withColumn('polarity', get_polarity_udf(F.col('cleaned_text')))\
        .withColumn('subjectivity', get_subjectivity_udf(F.col('cleaned_text')))

final_twitter_df.select("cleaned_text", "sentiment_vader", "polarity", "subjectivity").show()

In [ ]:
final_twitter_df = final_twitter_df.withColumn('polarity', get_polarity_udf(F.col('cleaned_text')))\
        .withColumn('subjectivity', get_subjectivity_udf(F.col('cleaned_text')))

In [ ]:
final_twitter_df.select("cleaned_text", "sentiment_vader", "polarity", "subjectivity").show()

In [ ]:
# create SQL view
final_twitter_df.createOrReplaceTempView("twitterSQL")

### 3.1 Volume of tweets 

In [ ]:
# select the relevant data
volume = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, COUNT(*) as volume \
                                    FROM twitterSQL \
                                    GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                                    ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
volume.show(100)

In [ ]:
# create SQL view
volume.createOrReplaceTempView("volumeSQL")

### 3.2 Average likes

We exclude tweets with 0 likes.

In [ ]:
# select the relevant data
avg_likes = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(favorite_count) as avg_likes \
                           FROM twitterSQL \
                           WHERE favorite_count > 0 \
                           GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                           ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_likes.show()

In [ ]:
# create SQL view
avg_likes.createOrReplaceTempView("avg_likesSQL")

### 3.3 Average Retweets

We exclude tweets with 0 retweets.

In [ ]:
# select the relevant data
avg_retweets = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(retweet_count) as avg_retweets \
                          FROM twitterSQL \
                          WHERE retweet_count > 0 \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_retweets.show()

In [ ]:
# create SQL view
avg_retweets.createOrReplaceTempView("avg_retweetsSQL")

### 3.4 Average Engagement rate

We define engagement rate of a tweet as the sum of likes and retweets divided by the amount of followers of the account that sent out the tweet. For our purpose we will take the avergage engagement rate per day. We exclude accounts who have no followers and we only take tweets into account which are liked and retweeted at least once.

In [ ]:
# select the relevant data
avg_engagement_rate = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(engagement_rate) as avg_engagement_rate \
                                     FROM twitterSQL \
                                     GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                                     ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_engagement_rate.show()

In [ ]:
# create SQL view
avg_engagement_rate.createOrReplaceTempView("avg_engagement_rateSQL")

### 3.5 Number of influencers

We will calculate how many influencers actively tweeted a certain day. We define an influencer as someone with:
- followers > 1000 
- engagement_rate > 0.20 
- weekly tweet frequency > 5

In [ ]:
def get_influencers(follower_count_tresh, eng_rate_tresh, freq_week_tresh, data):

    #df
    df = data
    
    # get all users with their amount of followers
    influencers = df.groupBy("screen_name") \
                    .agg(first("followers_count").alias("followers_count"))

    # average engagement rate for each user
    eng_rate = df.withColumn('eng_rate', ((df['favorite_count'] + df['retweet_count'])/df['followers_count']))

    eng_rate_user = eng_rate.groupBy("screen_name") \
                            .agg(avg("eng_rate").alias("eng_rate"))

    # average freq_weekly per user
    freq_week = df.withColumn("year", year(df["post_created_at"]))
    freq_week = freq_week.withColumn('week', weekofyear('post_created_at'))

    freq_week = freq_week.groupBy('screen_name', 'year', 'week').agg(countDistinct("full_text"))\
                    .withColumnRenamed("count(full_text)", "freq") \
                        .sort('screen_name', 'year', 'week', ascending = True)
    freq_week = freq_week.select('screen_name', 'freq')

    freq_week = freq_week.groupby("screen_name").agg(avg(freq_week.freq).alias('freq'))

    # put the data together
    data_joined = eng_rate_user.join(influencers, "screen_name").join(freq_week, "screen_name")

    # filter the data
    data_joined = data_joined.filter((data_joined.followers_count > follower_count_tresh) & (data_joined.eng_rate > eng_rate_tresh) & (data_joined.freq > freq_week_tresh))
    
    # show the data
    data_joined.show()
    return data_joined

In [ ]:
influencers = get_influencers(1000, 0.002, 2, final_twitter_df)

In [ ]:
# create SQL view
influencers.createOrReplaceTempView("influencersSQL")

In [ ]:
# select the relevant data
number_of_influencers = spark.sql(" SELECT DATE_FORMAT(a.post_created_at, 'Y-M-dd') as date, COUNT(b.screen_name) as influencers \
                                    FROM twitterSQL a \
                                    RIGHT OUTER JOIN influencersSQL b ON a.screen_name = b.screen_name\
                                    GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                                    ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
number_of_influencers.show()

In [ ]:
# create SQL view
number_of_influencers.createOrReplaceTempView("number_of_influencersSQL")

### 3.6 Average Followers 

In [ ]:
# select the relevant data
avg_followers = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(followers_count) as avg_followers \
                          FROM twitterSQL \
                          WHERE followers_count > 0 \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_followers.show()

In [ ]:
# create SQL view
number_of_influencers.createOrReplaceTempView("followersSQL")

### 3.7 Average Emoji Counts

In [ ]:
# select the relevant data
avg_emoji = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(emoji_count) as avg_emojis \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_emoji.show()

In [ ]:
# create SQL view
number_of_influencers.createOrReplaceTempView("emojiSQL")

### 3.8 Avergae Number of Words

In [ ]:
# select the relevant data
avg_words = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(num_words) as avg_words \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_words.show()

In [ ]:
# create SQL view
avg_words.createOrReplaceTempView("wordsSQL")

### 3.9 Avergae Number of Hashtags

In [ ]:
# select the relevant data
avg_hashtags = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(num_hashtags) as avg_hashtags \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_hashtags.show()

In [ ]:
# create SQL view
avg_hashtags.createOrReplaceTempView("hashtagsSQL")

### 3.10 Average Number of Mentions

In [ ]:
# select the relevant data
avg_mentions = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(num_mentions) as avg_mentions \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_mentions.show()

In [ ]:
# create SQL view
avg_mentions.createOrReplaceTempView("mentionsSQL")

### 3.11 Average Number of Exclamation Marks

In [ ]:
# select the relevant data
avg_marks = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(num_exclamation_marks) as avg_exclamation_marks \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_marks.show()

In [ ]:
# create SQL view
avg_marks.createOrReplaceTempView("marksSQL")

### 3.12 Sentiment 

In [ ]:
# select the relevant data
avg_sentiment = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(sentiment_vader) as avg_sentiment \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_sentiment.show()

In [ ]:
# create SQL view
avg_sentiment.createOrReplaceTempView("sentimentSQL")

### 3.13 Polarity

In [ ]:
# select the relevant data
avg_polarity = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(polarity) as avg_polarity \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_polarity.show()

In [ ]:
# create SQL view
avg_polarity.createOrReplaceTempView("polaritySQL")

### 3.13 Subjectivity

In [ ]:
# select the relevant data
avg_subjectivity = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(subjectivity) as avg_subjectivity \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_subjectivity.show()

In [ ]:
# create SQL view
avg_subjectivity.createOrReplaceTempView("subjectivitySQL")

## 4. Basetable

In [ ]:
# create basetable
basetable = volume.join(avg_likes, "date", how="inner") \
                    .join(avg_retweets, "date", how="inner") \
                    .join(avg_engagement_rate, "date", how="inner") \
                    .join(number_of_influencers, "date", how="inner") \
                    .join(avg_followers, "date", how="inner") \
                    .join(avg_emoji, "date", how="inner") \
                    .join(avg_words, "date", how="inner") \
                    .join(avg_hashtags, "date", how="inner") \
                    .join(avg_mentions, "date", how="inner") \
                    .join(avg_marks, "date", how="inner") \
                    .join(avg_sentiment, "date", how="inner") \
                    .join(avg_polarity, "date", how="inner") \
                    .join(avg_subjectivity, "date", how="inner") \
                    .join(trend, "date", how="left")

In [ ]:
# take a look at the basetable
basetable.show()

In [ ]:
basetable = basetable.toPandas()

In [ ]:
# export basetable as a .json file
basetable.to_json("./../../data/basetable_vegan_trend_prediction.json", orient="records", force_ascii=False, lines=True)

#### Read table in

In [26]:
# read in the saved basetable (.json)
basetable_df = spark.read.json("./../../data/basetable_vegan_trend_prediction.json")

In [27]:
basetable_df.show()

+------------+-------------------+---------------------+----------------+------------+-------------+------------+------------+-------------+-------------+----------------+-------------+----------+---------------+-----------+------+
|  avg_emojis|avg_engagement_rate|avg_exclamation_marks|   avg_followers|avg_hashtags|    avg_likes|avg_mentions|avg_polarity| avg_retweets|avg_sentiment|avg_subjectivity|    avg_words|      date|dependent_vegan|influencers|volume|
+------------+-------------------+---------------------+----------------+------------+-------------+------------+------------+-------------+-------------+----------------+-------------+----------+---------------+-----------+------+
|0.4280180762|       0.0303423202|         0.2091672046| 7811.1375488918| 1.506132989| 9.5102286402|1.2246610717|0.1127719889| 3.3629343629| 0.1322898644|    0.4252520269|25.9561007101|2021-11-03|            1.0|        120|  1549|
|0.8352165725|       0.0330735083|         0.3549905838| 3640.0143403442

In [28]:
# create SQL view
basetable_df.createOrReplaceTempView("basetableSQL")

In [55]:
check = spark.sql("SELECT date, volume,avg_retweets, dependent_vegan \
                  FROM basetableSQL"
                  )

In [56]:
check.show()

+----------+------+-------------+---------------+
|      date|volume| avg_retweets|dependent_vegan|
+----------+------+-------------+---------------+
|2021-11-03|  1549| 3.3629343629|            1.0|
| 2022-8-15|  1062|10.7292307692|            0.0|
| 2022-6-13|    88|      2.59375|            0.0|
| 2022-8-14|   906| 6.1367781155|            1.0|
| 2022-7-07|  1069| 6.7068062827|            0.0|
| 2022-5-26|   952| 6.2695924765|            1.0|
|2021-10-27|   396| 7.6951219512|            0.0|
| 2022-3-13|  1312|11.4625668449|            1.0|
| 2022-1-10|  1433| 5.5292929293|            0.0|
| 2022-7-15|   391|       10.312|            1.0|
| 2022-4-16|  1190| 6.7661290323|            1.0|
| 2022-5-01|   937| 7.6457680251|            1.0|
| 2022-8-03|  8537| 6.7667757774|            1.0|
| 2022-5-29|   786| 9.9288537549|            1.0|
| 2022-8-31|  1071| 8.1531531532|            1.0|
|2021-12-06|    36|        3.875|            0.0|
| 2022-9-07|  1164|  5.389380531|            0.0|


Look at the total number of observations in the basetable 

In [5]:
basetable_df.count()

223

## 5. split train and test set 

In [21]:
basetable_df.toPandas().head()

,avg_emojis,avg_engagement_rate,avg_exclamation_marks,avg_followers,avg_hashtags,avg_likes,avg_mentions,avg_polarity,avg_retweets,avg_sentiment,avg_subjectivity,avg_words,date,dependent_vegan,influencers,volume
0,0.428018,0.030342,0.209167,7811.137549,1.506133,9.510229,1.224661,0.112772,3.362934,0.132290,0.425252,25.956101,2021-11-03,1.0,120,1549
1,0.835217,0.033074,0.354991,3640.014340,6.118644,18.886054,0.565913,0.179490,10.729231,0.165516,0.415197,25.754237,2022-8-15,0.0,119,1062
2,0.579545,0.009035,0.352273,6038.816092,6.636364,5.222222,0.261364,0.240729,2.593750,0.191034,0.403160,24.806818,2022-6-13,0.0,4,88
3,0.985651,0.080885,0.349890,6134.959732,6.079470,14.191095,0.614790,0.169584,6.136778,0.152711,0.407482,25.415011,2022-8-14,1.0,89,906
4,0.830683,0.047927,0.423761,4375.890359,6.506080,12.462609,0.405051,0.214137,6.706806,0.178300,0.430801,25.509822,2022-7-07,0.0,76,1069


Look at the total number of observations in the basetable 

In [22]:
basetable_df.count()

223

We cannot use the randomsplit function, because we have time series data, so we have to use another approach
https://towardsdatascience.com/time-series-from-scratch-train-test-splits-and-evaluation-metrics-4fd654de1b37#:~:text=Train%2Ftest%20splits%20in%20time%20series%20In%20machine%20learning%2C,dataset%20for%20testing%20and%20everything%20else%20for%20training.

First we look at the amount of observations that will be assigned to the training set 

In [23]:
nr_train = int(basetable_df.count()*0.7)
nr_train

156

convert the final basetable to a pandas dataset 

In [24]:
basetable_pd = basetable_df.toPandas()
basetable_pd.head()

,avg_emojis,avg_engagement_rate,avg_exclamation_marks,avg_followers,avg_hashtags,avg_likes,avg_mentions,avg_polarity,avg_retweets,avg_sentiment,avg_subjectivity,avg_words,date,dependent_vegan,influencers,volume
0,0.428018,0.030342,0.209167,7811.137549,1.506133,9.510229,1.224661,0.112772,3.362934,0.132290,0.425252,25.956101,2021-11-03,1.0,120,1549
1,0.835217,0.033074,0.354991,3640.014340,6.118644,18.886054,0.565913,0.179490,10.729231,0.165516,0.415197,25.754237,2022-8-15,0.0,119,1062
2,0.579545,0.009035,0.352273,6038.816092,6.636364,5.222222,0.261364,0.240729,2.593750,0.191034,0.403160,24.806818,2022-6-13,0.0,4,88
3,0.985651,0.080885,0.349890,6134.959732,6.079470,14.191095,0.614790,0.169584,6.136778,0.152711,0.407482,25.415011,2022-8-14,1.0,89,906
4,0.830683,0.047927,0.423761,4375.890359,6.506080,12.462609,0.405051,0.214137,6.706806,0.178300,0.430801,25.509822,2022-7-07,0.0,76,1069


Removing date

In [10]:
basetable_pd = basetable_pd.drop(['date'], axis='columns')

In [20]:
basetable_pd.head(50)

,avg_emojis,avg_engagement_rate,avg_exclamation_marks,avg_followers,avg_hashtags,avg_likes,avg_mentions,avg_polarity,avg_retweets,avg_sentiment,avg_subjectivity,avg_words,dependent_vegan,influencers,volume
0,0.428018,0.030342,0.209167,7811.137549,1.506133,9.510229,1.224661,0.112772,3.362934,0.132290,0.425252,25.956101,1.0,120,1549
1,0.835217,0.033074,0.354991,3640.014340,6.118644,18.886054,0.565913,0.179490,10.729231,0.165516,0.415197,25.754237,0.0,119,1062
2,0.579545,0.009035,0.352273,6038.816092,6.636364,5.222222,0.261364,0.240729,2.593750,0.191034,0.403160,24.806818,0.0,4,88
3,0.985651,0.080885,0.349890,6134.959732,6.079470,14.191095,0.614790,0.169584,6.136778,0.152711,0.407482,25.415011,1.0,89,906
4,0.830683,0.047927,0.423761,4375.890359,6.506080,12.462609,0.405051,0.214137,6.706806,0.178300,0.430801,25.509822,0.0,76,1069
5,0.737395,0.025709,0.386555,5702.735889,5.657563,12.082051,0.497899,0.209101,6.269592,0.169099,0.425937,24.495798,1.0,78,952
6,0.439394,0.021345,0.209596,3704.725131,1.676768,14.878924,1.093434,0.127022,7.695122,0.126515,0.460762,29.348485,0.0,32,396
7,0.724848,0.033227,0.320884,5130.609509,4.293445,19.236439,0.586128,0.155153,11.462567,0.149604,0.411520,25.218750,1.0,129,1312
8,0.715980,0.028071,0.391486,9470.182910,5.845778,15.801001,0.515003,0.207223,5.529293,0.154718,0.443063,25.990230,0.0,93,1433
9,0.736573,0.020365,0.363171,4784.184143,5.948849,13.678414,0.370844,0.194724,10.312000,0.161826,0.421671,25.631714,1.0,40,391


Split the dataframe into train and test 

In [11]:
train_pd = basetable_pd.iloc[:nr_train,:]
test_pd = basetable_pd.iloc[nr_train:,:]

In [12]:
train_pd.head()

,avg_emojis,avg_engagement_rate,avg_exclamation_marks,avg_followers,avg_hashtags,avg_likes,avg_mentions,avg_polarity,avg_retweets,avg_sentiment,avg_subjectivity,avg_words,dependent_vegan,influencers,volume
0,0.428018,0.030342,0.209167,7811.137549,1.506133,9.510229,1.224661,0.112772,3.362934,0.132290,0.425252,25.956101,1.0,120,1549
1,0.835217,0.033074,0.354991,3640.014340,6.118644,18.886054,0.565913,0.179490,10.729231,0.165516,0.415197,25.754237,0.0,119,1062
2,0.579545,0.009035,0.352273,6038.816092,6.636364,5.222222,0.261364,0.240729,2.593750,0.191034,0.403160,24.806818,0.0,4,88
3,0.985651,0.080885,0.349890,6134.959732,6.079470,14.191095,0.614790,0.169584,6.136778,0.152711,0.407482,25.415011,1.0,89,906
4,0.830683,0.047927,0.423761,4375.890359,6.506080,12.462609,0.405051,0.214137,6.706806,0.178300,0.430801,25.509822,0.0,76,1069


In [16]:
basetable_pd['dependent_vegan'].isnull().values.sum()

6

Convert the pandas dataframe back to a spark dataframe

In [13]:
train = spark.createDataFrame(train_pd)
train.show()

Py4JError: An error occurred while calling o42.legacyInferArrayTypeFromFirstElement. Trace:
py4j.Py4JException: Method legacyInferArrayTypeFromFirstElement([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:1589)



In [ ]:
test = spark.createDataFrame(test_pd)
test.show()

In [ ]:
# get number in observations in each set
print("Number of observations train: %s" %train.count())
print("Number of observations test: %s" %test.count())

In [ ]:
# get distribution of dependent variable within each set
train.groupBy("dependent_vegan").count().show()
test.groupBy("dependent_vegan").count().show()

In [57]:
train = train.na.drop()
test = test.na.drop()

NameError: name 'train' is not defined

# 6. Modelling

## 6.1 Pipelines

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.feature import Tokenizer, StopWordsRemover, Word2Vec
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [ ]:
# define indexer (IDX)
IDX = StringIndexer(inputCol="dependent_vegan", outputCol="label")

In [ ]:
train.printSchema()

In [ ]:
# define all the numeric features
numFeatureCols = ["volume", "avg_likes", "avg_retweets", "avg_engagement_rate", "avg_followers", "avg_emojis", "avg_words", "avg_hashtags", "avg_mentions", "avg_exclamation_marks", "avg_sentiment"]

# define vector assembler
VA_num = VectorAssembler(inputCols=numFeatureCols, outputCol="numeric_features")

In [ ]:
# define the standard scaler (SS)
SS = StandardScaler(inputCol="numeric_features", outputCol="scaled_numeric_features")

In [ ]:
# for random forest we don't scale the features 
VA_RF = VectorAssembler(inputCols=["volume", "avg_likes", "avg_retweets", "avg_engagement_rate", "avg_followers", "avg_emojis", "avg_words", "avg_hashtags", "avg_mentions", "avg_exclamation_marks", "avg_sentiment", "avg_polarity", "avg_subjectivity"], outputCol="featuresrf")

In [ ]:
# define vector assembler (VA_all)
VA_all = VectorAssembler(inputCols=["scaled_numeric_features", "avg_polarity", "avg_subjectivity"], outputCol="features")

In [ ]:
# define logistic regression model
LR = LogisticRegression(featuresCol = "features", labelCol = "label")

In [ ]:
# define decision tree model
DT = DecisionTreeClassifier(featuresCol = "features", labelCol = "label")

In [ ]:
# define decision tree model
GBT = GBTClassifier(featuresCol = "features", labelCol = "label")

In [ ]:
# define random forest model
RF = RandomForestClassifier(featuresCol = "featuresrf", labelCol = "label", numTrees= 200, featureSubsetStrategy= 'all', maxDepth= 10)

## 6.2 Modelling

In [ ]:
# define pipeline model and fit on training data
LR_pipeline = Pipeline().setStages([IDX, VA_num, SS, VA_all, LR]).fit(train)

In [ ]:
# get predictions on test set
LR_preds = LR_pipeline.transform(test)

In [ ]:
# define pipeline model and fit on training data
DT_pipeline = Pipeline().setStages([IDX, VA_num, SS, VA_all, DT]).fit(train)

In [ ]:
# get predictions on test set
DT_preds = DT_pipeline.transform(test)

In [ ]:
# define pipeline model and fit on training data
GBT_pipeline = Pipeline().setStages([IDX, VA_num, SS, VA_all, GBT]).fit(train)

In [ ]:
# get predictions on test set
GBT_preds = GBT_pipeline.transform(test)

In [ ]:
# define pipeline model and fit on training data
RF_pipeline = Pipeline().setStages([IDX, VA_RF, RF]).fit(train)

In [ ]:
# get predictions on test set
RF_preds = RF_pipeline.transform(test)

# 7. Model Evaluation

In [ ]:
# import packages
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# define evaluator (for AUC)
evaluator_auc = BinaryClassificationEvaluator()

# define evaluator (for other metrics)
evaluator_mc = MulticlassClassificationEvaluator()

In [ ]:
# get metrics for LR model
lr_f1 = evaluator_mc.evaluate(LR_preds, {evaluator_mc.metricName: "f1"})
lr_accuracy = evaluator_mc.evaluate(LR_preds, {evaluator_mc.metricName: "accuracy"})
lr_recall = evaluator_mc.evaluate(LR_preds, {evaluator_mc.metricName: "recallByLabel"})
lr_auc = evaluator_auc.evaluate(LR_preds, {evaluator_auc.metricName: 'areaUnderROC'})

In [ ]:
# get metrics for DT model
dt_f1 = evaluator_mc.evaluate(DT_preds, {evaluator_mc.metricName: "f1"})
dt_accuracy = evaluator_mc.evaluate(DT_preds, {evaluator_mc.metricName: "accuracy"})
dt_recall = evaluator_mc.evaluate(DT_preds, {evaluator_mc.metricName: "recallByLabel"})
dt_auc = evaluator_auc.evaluate(DT_preds, {evaluator_auc.metricName: 'areaUnderROC'})

In [ ]:
# get metrics for GBT model
gbt_f1 = evaluator_mc.evaluate(GBT_preds, {evaluator_mc.metricName: "f1"})
gbt_accuracy = evaluator_mc.evaluate(GBT_preds, {evaluator_mc.metricName: "accuracy"})
gbt_recall = evaluator_mc.evaluate(GBT_preds, {evaluator_mc.metricName: "recallByLabel"})
gbt_auc = evaluator_auc.evaluate(GBT_preds, {evaluator_auc.metricName: 'areaUnderROC'})

In [ ]:
# get metrics for RF model
rf_f1 = evaluator_mc.evaluate(RF_preds, {evaluator_mc.metricName: "f1"})
rf_accuracy = evaluator_mc.evaluate(RF_preds, {evaluator_mc.metricName: "accuracy"})
rf_recall = evaluator_mc.evaluate(RF_preds, {evaluator_mc.metricName: "recallByLabel"})
rf_auc = evaluator_auc.evaluate(RF_preds, {evaluator_auc.metricName: 'areaUnderROC'})

In [ ]:
# check which of both algorithms is the best:
print("LOGISTIC REGRESSION:")
print('  F1       : %g' % lr_f1)
print('  ACCURACY : %g' % lr_accuracy)
print('  RECALL   : %g' % lr_recall)
print('  AUC      : %g' % lr_auc)
print("------------------")
print("SINGLE DECISION TREE:")
print('  F1       : %g' % dt_f1)
print('  ACCURACY : %g' % dt_accuracy)
print('  RECALL   : %g' % dt_recall)
print('  AUC      : %g' % dt_auc)
print("------------------")
print("GRADIENT-BOOSTED TREES:")
print('  F1       : %g' % gbt_f1)
print('  ACCURACY : %g' % gbt_accuracy)
print('  RECALL   : %g' % gbt_recall)
print('  AUC      : %g' % gbt_auc)
print("------------------")
print("RANDOM FOREST:")
print('  F1       : %g' % rf_f1)
print('  ACCURACY : %g' % rf_accuracy)
print('  RECALL   : %g' % rf_recall)
print('  AUC      : %g' % rf_auc)

# 8 Plot model evaluation

## 8.1 Confusion Matrix 

We plot the confussion matrix for the model with the highest AUC, this is the random forest model with cross validation

In [ ]:
# import confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# get predictions and labels
preds_and_labels = RF_preds.select(['prediction','label']) \
                                  .withColumn('label', F.col('label').cast(FloatType())) \
                                  .orderBy('prediction') \
                                  .toPandas()

# get confusion matrix
cm = confusion_matrix(preds_and_labels["label"], preds_and_labels["prediction"], labels=[0, 1])
# get confusion matrix figure
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
# plot figure
disp.plot()
plt.show()

Calculate sensitivity and specificity

In [ ]:
from sklearn.metrics import roc_curve, auc, recall_score
recall_sensitivity = metrics.recall_score(preds_and_labels["label"], preds_and_labels["prediction"], pos_label=1)
recall_specificity = metrics.recall_score(preds_and_labels["label"], preds_and_labels["prediction"], pos_label=0)
recall_sensitivity, recall_specificity 

## 8.2 Model interpretation

We plot the future importance for the model with the highest AUC, this is the random forest model

In [ ]:
# inspect feature importance
feature_importance = RF_pipeline.stages[-1].featureImportances.toArray()
# define all the features
all_feature_names = numFeatureCols + ["avg_polarity", "avg_subjectivity"] 
plt.figure(figsize=(20, 5))
plt.bar(x=range(len(feature_importance)), height=feature_importance)
plt.xticks(range(len(feature_importance)), all_feature_names, rotation=90)
plt.show()

## 8.3 ROC Curve

We will plot the ROC curve for the model with the highest AUC, this is the random forest model.

We will use plotty to plot the ROC curve, so we will have to perform some data type transformations 

In [ ]:
# convert the rf predictions to a pandas dataframe
RF_preds_pd = RF_preds.toPandas()
RF_preds_pd.head()

In [ ]:
# this is a function that will select the probabilities for value 1
def select1(column):
    one = column[1]
    return one 

In [ ]:
# we apply this function to our probabilities 
Yscore = RF_preds_pd['probability'].apply(lambda x: select1(x)).to_numpy()


In [ ]:
# Yscore contains the probabilities of our predictions to be 1
Yscore

In [ ]:
# Yscore is of the correct type to perform the plot 
type(Yscore)

In [ ]:
# Y is the real value of our dependent
Y = test.select('dependent_vegan').toPandas()
Y = Y['dependent_vegan'].to_numpy()
Y

In [ ]:
# Yscore is of the correct type to perform the plot 
type(Y)

In [ ]:
# we use the roc_curve function to predict the false-positive, true-positive rate and thresholds based on the Y an Y score
fpr, tpr, thresholds = roc_curve(Y, Yscore)

In [ ]:
fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()